In [1]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

In [2]:
%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D

import torch.nn.functional as F
import torch.optim as optim
from torch.utils import data

from tqdm import tqdm
from sklearn import datasets
import random

In [3]:
import mylibrary.datasets as datasets

In [4]:
device = torch.device("cuda:0")
# device = torch.device("cuda:1")
# device = torch.device("cpu")

In [5]:
mnist = datasets.FashionMNIST()
# mnist.download_mnist()
# mnist.save_mnist()
train_data, train_label_, test_data, test_label_ = mnist.load()

train_data = train_data / 255.
test_data = test_data / 255.

train_size = len(train_label_)

In [6]:
## converting data to pytorch format
train_data = torch.Tensor(train_data)
test_data = torch.Tensor(test_data)
train_label = torch.LongTensor(train_label_)
test_label = torch.LongTensor(test_label_)

In [7]:
## converting data to pytorch format
train_data = torch.Tensor(train_data)
test_data = torch.Tensor(test_data)
train_label = torch.LongTensor(train_label_)

In [8]:
input_size = 784
output_size = 10

In [9]:
class MNIST_Dataset(data.Dataset):
    
    def __init__(self, data, label):
        self.data = data
        self.label = label
        
#         self.label = mask.type(torch.float32).reshape(-1,1)
        self._shuffle_data_()
        
    def __len__(self):
        return len(self.data)
    
    def _shuffle_data_(self):
        randidx = random.sample(range(len(self.data)), k=len(self.data))
        self.data = self.data[randidx]
        self.label = self.label[randidx]
    
    def __getitem__(self, idx):
        img, lbl = self.data[idx], self.label[idx]
        return img, lbl

In [10]:
train_dataset = MNIST_Dataset(train_data, train_label)
test_dataset = MNIST_Dataset(test_data, test_label)

In [11]:
learning_rate = 0.0003
batch_size = 50

In [12]:
train_loader = data.DataLoader(dataset=train_dataset, num_workers=4, batch_size=batch_size, shuffle=True)
test_loader = data.DataLoader(dataset=test_dataset, num_workers=4, batch_size=batch_size, shuffle=False)

## Making of matrix factorized layer

In [13]:
class PairWeight(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        assert input_dim%2 == 0, "Input dim must be even number"
        self.weight = torch.eye(2).unsqueeze(0).repeat_interleave(input_dim//2, dim=0)
        self.weight = nn.Parameter(self.weight)
        
    def forward(self, x):
        bs, dim = x.shape[0], x.shape[1]
        x = x.view(bs, -1, 2).transpose(0,1)
        x = torch.bmm(x, self.weight)
        x = x.transpose(1,0).reshape(bs, -1)
        return x

In [17]:
pairW = PairWeight(784).to(device)
pairW_s = torch.jit.script(pairW)
_a = torch.randn(100, 784).to(device)

In [18]:
%timeit pairW(_a) 

1.18 ms ± 553 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [19]:
%timeit pairW_s(_a) 

1.18 ms ± 914 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [14]:
# class PairWeight(nn.Module):
#     def __init__(self, input_dim):
#         super().__init__()
#         assert input_dim%2 == 0, "Input dim must be even number"
#         self.weight = torch.eye(2).unsqueeze(0).repeat_interleave(input_dim//2, dim=0)
#         self.weight = nn.Parameter(self.weight)
        
#     def forward(self, x):
#         bs = x.shape[0]
#         x = x.view(-1, 2)
#         wi = torch.arange(self.weight.shape[0]).repeat(bs)
#         W = self.weight[wi]
#         x = torch.bmm(x.unsqueeze(1), W).view(bs, -1)
#         return x

In [47]:
class PairFactorizedLinear(nn.Module):
    
    def __init__(self, input_dim, bias=True):
        super().__init__()
        assert input_dim%2 == 0, "Input dim must be even number"
        self.input_dim = input_dim
        
        num_layers = int(np.ceil(np.log2(input_dim)))
            
        self.facto_nets = []
        self.idx_revidx = []
        for i in range(num_layers):
            idrid = self.get_pair(self.input_dim, i+1)
            net = PairWeight(self.input_dim)
            self.facto_nets.append(net)
            self.idx_revidx.append(idrid)
            
#         self.facto_nets = nn.ModuleList(self.facto_nets)
        
        self.fn0 = self.facto_nets[0]
        self.facto_nets = nn.ModuleList(self.facto_nets[1:])
        
        self.bias = None
        if bias: self.bias = nn.Parameter(torch.zeros(self.input_dim))
            
    def get_pair(self, inp_dim, step=1):
        dim = 2**int(np.ceil(np.log2(inp_dim)))
        assert isinstance(step, int), "Step must be integer"

        blocks = (2**step)
        range_ = dim//blocks
        adder_ = torch.arange(0, range_)*blocks

        pairs_ = torch.Tensor([0, blocks//2])
        repeat_ = torch.arange(0, blocks//2).reshape(-1,1)
        block_map = (pairs_+repeat_).reshape(-1)

        reorder_for_pair = (block_map+adder_.reshape(-1,1)).reshape(-1)
        indx = reorder_for_pair.type(torch.long)
        indx = indx[indx<inp_dim]

        rev_indx = torch.argsort(indx)
        return indx, rev_indx
    
    def forward(self, x):
        ## swap first and then forward and reverse-swap
#         y = self.facto_nets[0](x)
        y = self.fn0(x)

#         for i in range(1, len(self.facto_nets)):
#             idx, revidx = self.idx_revidx[i]
#             y = self.facto_nets[i](y[:, idx])[:, revidx]
        for i, fn in enumerate(self.facto_nets, 1):
            idx, revidx = self.idx_revidx[i]
            y = y[:, idx]
            y = fn(y)
            y = y[:, revidx]

        if self.bias is not None: 
            y = y+self.bias
        return y

In [48]:
pfL = PairFactorizedLinear(784).to(device)

pfL_s = torch.jit.script(pfL)
_a = torch.randn(100, 784).to(device)

In [49]:
# pfL(torch.randn(2, 100))

In [52]:
%timeit pfL(_a) 

13.5 ms ± 174 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [53]:
%timeit pfL_s(_a) 

13.2 ms ± 130 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [18]:
param_count = sum([torch.numel(p) for p in pfL.parameters()])
param_count

1500

In [19]:
100*100/param_count

6.666666666666667

In [20]:
# class FactorNet(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.la1 = PairFactorizedLinear(784, bias=False)
#         self.bn1 = nn.BatchNorm1d(784)
#         self.la2 = PairFactorizedLinear(784, bias=False)
#         self.bn2 = nn.BatchNorm1d(784)
#         self.la3 = nn.Linear(784, 10)
        
#     def forward(self, x):
#         x = self.bn1(self.la1(x))
#         x = torch.relu(x)
#         x = self.bn2(self.la2(x))
#         x = torch.relu(x)
#         x = self.la3(x)
#         return x

In [21]:
class FactorNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.la1 = PairFactorizedLinear(784, bias=False)
        self.bn1 = nn.BatchNorm1d(784)
        self.la2 = PairFactorizedLinear(784, bias=False)
        self.bn2 = nn.BatchNorm1d(784)
        self.la3 = PairFactorizedLinear(784, bias=False)
        self.bn3 = nn.BatchNorm1d(784)
        self.la4 = PairFactorizedLinear(784, bias=False)
        self.bn4 = nn.BatchNorm1d(784)
        self.fc = nn.Linear(784, 10)
        
    def forward(self, x):
        x = self.bn1(self.la1(x))
        x = torch.relu(x)
        x = self.bn2(self.la2(x))
        x = torch.relu(x)
        x = self.bn3(self.la3(x))
        x = torch.relu(x)
        x = self.bn4(self.la4(x))
        x = torch.relu(x)
        x = self.fc(x)
        return x

In [22]:
class OrdinaryNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.la1 = nn.Linear(784, 784, bias=False)
        self.bn1 = nn.BatchNorm1d(784)
        self.la2 = nn.Linear(784, 784, bias=False)
        self.bn2 = nn.BatchNorm1d(784)
        self.la3 = nn.Linear(784, 10)
        
    def forward(self, x):
        x = self.bn1(self.la1(x))
        x = torch.relu(x)
        x = self.bn2(self.la2(x))
        x = torch.relu(x)
        x = self.la3(x)
        return x

In [23]:
model = FactorNet()
param_count = sum([torch.numel(p) for p in model.parameters()])
param_count

76842

In [24]:
model = OrdinaryNet()
param_count1 = sum([torch.numel(p) for p in model.parameters()])
param_count1, param_count1/param_count

(1240298, 16.14088649436506)

### Model Development

In [25]:
torch.manual_seed(0)
model = FactorNet().to(device)
# model = OrdinaryNet().to(device)
model

FactorNet(
  (la1): PairFactorizedLinear(
    (facto_nets): ModuleList(
      (0): PairWeight()
      (1): PairWeight()
      (2): PairWeight()
      (3): PairWeight()
      (4): PairWeight()
      (5): PairWeight()
      (6): PairWeight()
      (7): PairWeight()
      (8): PairWeight()
      (9): PairWeight()
    )
  )
  (bn1): BatchNorm1d(784, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (la2): PairFactorizedLinear(
    (facto_nets): ModuleList(
      (0): PairWeight()
      (1): PairWeight()
      (2): PairWeight()
      (3): PairWeight()
      (4): PairWeight()
      (5): PairWeight()
      (6): PairWeight()
      (7): PairWeight()
      (8): PairWeight()
      (9): PairWeight()
    )
  )
  (bn2): BatchNorm1d(784, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (la3): PairFactorizedLinear(
    (facto_nets): ModuleList(
      (0): PairWeight()
      (1): PairWeight()
      (2): PairWeight()
      (3): PairWeight()
      (4): PairWeight()
      

In [26]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0003)

In [27]:
print("number of params: ", sum(p.numel() for p in model.parameters()))

number of params:  76842


In [28]:
losses = []
train_accs = []
test_accs = []
EPOCHS = 20

for epoch in range(EPOCHS):
    
    train_acc = 0
    train_count = 0
    for xx, yy in tqdm(train_loader):
        xx, yy = xx.to(device), yy.to(device)

        yout = model(xx)
        loss = criterion(yout, yy)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        losses.append(float(loss))

        outputs = torch.argmax(yout, dim=1).data.cpu().numpy()
        correct = (outputs == yy.data.cpu().numpy()).astype(float).sum()
        train_acc += correct
        train_count += len(outputs)

    train_accs.append(float(train_acc)/train_count*100)
    train_acc = 0
    train_count = 0

    print(f'Epoch: {epoch},  Loss:{float(loss)}')
    test_count = 0
    test_acc = 0
    for xx, yy in tqdm(test_loader):
        xx, yy = xx.to(device), yy.to(device)
        with torch.no_grad():
            yout = model(xx)
        outputs = torch.argmax(yout, dim=1).data.cpu().numpy()
        correct = (outputs == yy.data.cpu().numpy()).astype(float).sum()
        test_acc += correct
        test_count += len(xx)
    test_accs.append(float(test_acc)/test_count*100)
    print(f'Train Acc:{train_accs[-1]:.2f}%, Test Acc:{test_accs[-1]:.2f}%')
    print()

### after each class index is finished training
print(f'\t-> Train Acc {max(train_accs)} ; Test Acc {max(test_accs)}')

  0%|          | 0/200 [00:00<?, ?it/s]

Epoch: 0,  Loss:0.4311772584915161


  0%|          | 0/1200 [00:00<?, ?it/s]

Train Acc:78.84%, Test Acc:83.83%



  0%|          | 0/200 [00:00<?, ?it/s]

Epoch: 1,  Loss:0.2603623867034912


  0%|          | 0/1200 [00:00<?, ?it/s]

Train Acc:86.09%, Test Acc:85.78%



 84%|████████▍ | 1012/1200 [00:22<00:04, 45.59it/s]


KeyboardInterrupt: 

In [ ]:
## stats: 20 epochs
### lin/factlin+bn+relu+linear
### for factor-net -> 42346 -> Train Acc 89.945 ; Test Acc 87.43
### for ordinary-net -> 1240298 -> Train Acc 97.81333333333333 ; Test Acc 89.66


In [ ]:
### for factor-net : factlin+bn+relu+factlin+bn+relu+linear->  76842
###	-> Train Acc 94.33834 ; Test Acc 88.67